In [1]:
import numpy as np
import torch
import os
import pickle
import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams['figure.figsize'] = (20, 20)
# plt.rcParams['image.interpolation'] = 'bilinear'

from argparse import ArgumentParser

from torch.optim import SGD, Adam
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Normalize
from torchvision.transforms import ToTensor, ToPILImage
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
import torch.nn as nn
from torch.optim import lr_scheduler
from networks.SegUNet_old_version import SegUNet_old_version

import collections
import numbers
import random
import math
from PIL import Image, ImageOps, ImageEnhance
import logging
import time
import tool
# import bcolz

In [2]:
CROSS_VALIDATION_FOLD = 0 # 0-4
SEED = CROSS_VALIDATION_FOLD * 100
NUM_CHANNELS = 5
NUM_CLASSES = 2 
model_name = 'SegUNet_old_version_position_5channels'
log_path = 'log/'
save_weights_path = '../_weights/'
if not os.path.exists(save_weights_path):
    os.makedirs(save_weights_path)
if not os.path.exists(log_path):
    os.makedirs(log_path)

In [3]:
log_filename = log_path + model_name + '-fold'+str(CROSS_VALIDATION_FOLD)+'.log'
logging.basicConfig(filename=log_filename, level=logging.INFO, 
                   format='%(asctime)s:%(levelname)s:%(message)s')
def log(message):
    print(message)
    logging.info(message)

In [4]:
log('='*50 + 'start run' + '='*50)

==================================================start run==================================================


## dataset

In [ ]:
xx,yy,zz = np.meshgrid(np.linspace(normstart[0],normstart[0]+normsize[0],self.crop_size[0]/self.stride),
                   np.linspace(normstart[1],normstart[1]+normsize[1],self.crop_size[1]/self.stride),
                   np.linspace(normstart[2],normstart[2]+normsize[2],self.crop_size[2]/self.stride),indexing ='ij')
coord = np.concatenate([xx[np.newaxis,...], yy[np.newaxis,...],zz[np.newaxis,:]],0).astype('float32')

In [5]:
a = np.array([-0.5, 0.5]).astype('float32')

In [6]:
a

array([-0.5,  0.5], dtype=float32)

In [5]:
# NUM_CHANNELS = 3
# NUM_CLASSES = 2 # car is 1, background is 0

# color_transform = Colorize(n=NUM_CLASSES)
# image_transform = ToPILImage()

In [6]:
class Crop_different_size_for_image_and_label(object):
    def __init__(self, image_size=572, label_size=388):
        self.image_size = image_size
        self.label_size = label_size
        self.bound = (self.image_size - self.label_size) // 2

    def __call__(self, img_and_label):
        img, label = img_and_label
        w, h = img.size
        
        xcenter = 1000
        ycenter = 600
        
        img = img.crop((xcenter - self.image_size // 2, ycenter - self.image_size // 2, xcenter + self.image_size // 2, ycenter + self.image_size // 2))
        label = label.crop((xcenter - self.label_size // 2, ycenter - self.label_size // 2, xcenter + self.label_size // 2, ycenter + self.label_size // 2))
        
        return img, label

In [7]:
# random_rotate = tool.Random_Rotate_Crop(maxAngle = 10)
# crop = tool.Random_Rotate_Crop(maxAngle = 0)
crop_512 = tool.RandomCrop(crop_size = 512)
# crop_572 = tool.RandomCrop_different_size_for_image_and_label(image_size=572, label_size=388)
# random_color = tool.RandomColor()
to_tensor_label = tool.ToTensor_Label()
normalize = tool.ImageNormalize([.485, .456, .406], [.229, .224, .225])
train_transforms = tool.Compose([crop_512, to_tensor_label, normalize])
val_transforms = tool.Compose([crop_512, to_tensor_label, normalize])

In [8]:
image_path = '../../data/images/train/'
mask_path = '../../data/images/train_masks/'

In [9]:
with open('./train_shuffle_names.pk', 'rb') as f:
    filenames = pickle.load(f)

In [10]:
fold_num = len(filenames) // 5
folds = []
for i in range(5):
    if i == 4:
        folds.append(filenames[i * fold_num :])
    else:
        folds.append(filenames[i * fold_num : (i + 1) * fold_num])

train_filenames = []
for i in range(5):
    if i == CROSS_VALIDATION_FOLD:
        val_filenames = folds[i]
    else:
        train_filenames += folds[i]

In [11]:
# train_filenames = train_filenames[:4]
# val_filenames = val_filenames[:1]

In [12]:
train_set = tool.Car_dataset(image_path, mask_path, train_filenames, train_transforms, ifFlip=True) 
val_set = tool.Car_dataset(image_path, mask_path, val_filenames, val_transforms, ifFlip=True) # for validation set
train_loader = DataLoader(train_set, num_workers=4, batch_size=4, shuffle=True)
val_loader = DataLoader(val_set, num_workers=4, batch_size=1) # for validation set

## dataset examples

In [13]:
# inp, tar = train_loader.__iter__().next()

In [14]:
# i = 0
# inp = Variable(inp)
# tar = Variable(tar)
# # tar[:, 0]
# t = tar[i].cpu().data.numpy()
# inpu = inp[i].cpu().data.numpy()

In [15]:
# img_tensor = inp[i]
# for ten, m, s in zip(img_tensor, [.229, .224, .225], [.485, .456, .406]):
#     ten.mul_(m).add_(s)
# img = ToPILImage()(img_tensor)

In [16]:
# img

In [17]:
# b = (572-388)//2

In [18]:
# img = img.crop((b, b, b + 388, b + 388))

In [19]:
# img

In [20]:
# label = Image.fromarray(t[0].astype('uint8') * 255)
# label

## train

In [21]:
def load_model(filename, model, optimizer):
    checkpoint = torch.load(filename)
    model.load_state_dict(checkpoint['model_state'])
    optimizer.load_state_dict(checkpoint['optimizer_state'])
    
def save_model(filename, model, optimizer):
    torch.save({'model_state': model.state_dict(),
                'optimizer_state': optimizer.state_dict()}, 
                filename)

In [22]:
class CrossEntropyLoss2d(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(CrossEntropyLoss2d, self).__init__()
        self.loss = nn.NLLLoss(weight, size_average)

    def forward(self, outputs, targets):
        return self.loss(F.log_softmax(outputs, dim=1), targets)

In [23]:
def train(epoch, steps_plot=0):
    model.train()

    weight = torch.ones(NUM_CLASSES)
    criterion = CrossEntropyLoss2d(weight.cuda()) # loss function

    epoch_loss = []
    step_loss = []

    for step, (images, labels) in enumerate(train_loader):
        
        images = images.cuda()
        labels = labels.cuda()

        inputs = Variable(images)
        targets = Variable(labels)
        outputs = model(inputs)

        optimizer.zero_grad()
        loss = criterion(outputs, targets[:, 0])
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())
        step_loss.append(loss.item())
        
        if step % 10 == 0:
            average_step_loss = sum(step_loss) / len(step_loss)
            message = 'Epoch[{}]({}/{}): \tloss: {:.4}'.format(epoch, step, len(train_loader), average_step_loss)
            log(message)
            step_loss = []
    average = sum(epoch_loss) / len(epoch_loss)
    message = 'Train: Epoch[{}] \taverage loss: {:.4}'.format(epoch, average)
    log(message)

In [24]:
def test(steps_plot = 0):
    model.eval()

    weight = torch.ones(NUM_CLASSES)
    criterion = CrossEntropyLoss2d(weight.cuda())

#     for epoch in range(start_epoch, end_epochs+1):
    total_loss = []

    for step, (images, labels) in enumerate(val_loader):

        images = images.cuda()
        labels = labels.cuda()

        inputs = Variable(images)
        targets = Variable(labels)
        outputs = model(inputs)

        loss = criterion(outputs, targets[:, 0])

        total_loss.append(loss.item())

    average = sum(total_loss) / len(total_loss)
    message = 'Validation: \taverage loss: {:.4}'.format(average)
    log(message)
    return average

## train


In [26]:
torch.cuda.manual_seed_all(SEED)
model = SegUNet(in_channels=NUM_CHANNELS, n_classes=NUM_CLASSES)
model = model.cuda()
optimizer = Adam(model.parameters(), lr = 1e-3)
# load_model('save_model_test.pth', model, optimizer)

In [27]:
save_weights_path = '../_weights/'
scheduler = lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.5)
val_losses = []
start_time = time.time()
epoch_num = 250
for epoch in range(epoch_num):
    scheduler.step(epoch)
    message = 'learning rate: ' + str(scheduler.get_lr()[0])
    log(message)
    train(epoch)
    log('-'*100)
    
    if epoch == 0:
        t1 = time.time()
        message = 'one epoch time: ' + str(t1 - start_time) + 's'
        log(message)
        log('-'*100)
        
    val_loss = test()
    log('-'*100)
    
    val_losses.append(val_loss)
    if val_loss == min(val_losses) and epoch >= 100:
        save_file_name = save_weights_path+model_name+'-fold'+str(CROSS_VALIDATION_FOLD)+'-%.5f' % val_loss+'.pth'
        save_model(save_file_name, model, optimizer)
        
end_time = time.time()
total_time = end_time - start_time
average_time = total_time / epoch_num
message = 'total_time: ' + str(total_time) + 's' + '\t\taverage_time: ' + str(average_time) + 's'
log(message)

learning rate: 0.001


RuntimeError: indices and input shapes do not match: indices [4 x 256 x 64 x 64], input [4 x 512 x 64 x 64] at /pytorch/aten/src/THCUNN/generic/SpatialMaxUnpooling.cu:15